In [1]:
import gymnasium as gym
env = gym.make("Walker2d-v4")
env.reset()
print("Mujoco environment initialized successfully!")

/Users/huangfei/.pyenv/versions/3.12.0/lib/python3.12/site-packages/gymnasium/envs/registration.py:517: DeprecationWarning: WARN: The environment Walker2d-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


Mujoco environment initialized successfully!


In [2]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# 创建 Walker2d 环境
env = gym.make("Walker2d-v4")

# 初始化 PPO 代理
model = PPO("MlpPolicy", env, verbose=1)

# 训练 100k 步
model.learn(total_timesteps=100000)

# 评估代理性能
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std: {std_reward}")

# 保存模型
model.save("walker2d_ppo")

/Users/huangfei/.pyenv/versions/3.12.0/lib/python3.12/site-packages/gymnasium/envs/registration.py:517: DeprecationWarning: WARN: The environment Walker2d-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 18.8     |
|    ep_rew_mean     | -0.0331  |
| time/              |          |
|    fps             | 1654     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 21.3       |
|    ep_rew_mean          | 2.65       |
| time/                   |            |
|    fps                  | 1105       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01829367 |
|    clip_fraction        | 0.243      |
|    clip_range           | 0.2        |
|    entropy_loss         | -8.52

/Users/huangfei/.pyenv/versions/3.12.0/lib/python3.12/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: 260.9132532954216, Std: 3.0435169169062126


In [4]:
import time
import gymnasium as gym
from stable_baselines3 import PPO

# 创建可视化环境
env = gym.make("Walker2d-v5", render_mode="human")  # 改成 v5 以保持最新
model = PPO.load("walker2d_ppo")

obs, _ = env.reset()
done = False
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, _ = env.step(action)
    env.render()
    time.sleep(0.01)

env.close()

/Users/huangfei/.pyenv/versions/3.12.0/lib/python3.12/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


In [5]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO

# 加载训练好的模型
model = PPO.load("walker2d_ppo")

# 创建 Walker2d 环境
env = gym.make("Walker2d-v5")

# 评估 Walker2d 代理
n_episodes = 10  # 评估 10 轮
episode_rewards = []
episode_lengths = []

for i in range(n_episodes):
    obs, _ = env.reset()
    done = False
    total_reward = 0
    steps = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _, _ = env.step(action)
        total_reward += reward
        steps += 1

    episode_rewards.append(total_reward)
    episode_lengths.append(steps)

# 计算评估结果
mean_reward = np.mean(episode_rewards)
std_reward = np.std(episode_rewards)
mean_length = np.mean(episode_lengths)

print(f"🏆 Walker2d 训练评估结果（基于 {n_episodes} 轮测试）：")
print(f"✅ 平均奖励（Mean Reward）: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"✅ 平均步数（Mean Episode Length）: {mean_length:.2f} steps")
print(f"✅ 单轮最大奖励（Max Episode Reward）: {np.max(episode_rewards):.2f}")
print(f"✅ 单轮最小奖励（Min Episode Reward）: {np.min(episode_rewards):.2f}")

# 判断 Walker 是否训练成功
if mean_reward > 200:
    print("🎉 Walker2d 训练效果良好！Walker 能较稳定地行走。")
elif mean_reward > 100:
    print("⚠️ Walker2d 训练一般，可能偶尔摔倒或动作不自然，可以考虑增加训练步数。")
else:
    print("❌ Walker2d 训练较差，Walker 可能不能行走，建议调整超参数或训练更久。")


🏆 Walker2d 训练评估结果（基于 10 轮测试）：
✅ 平均奖励（Mean Reward）: 264.47 ± 5.28
✅ 平均步数（Mean Episode Length）: 146.80 steps
✅ 单轮最大奖励（Max Episode Reward）: 273.22
✅ 单轮最小奖励（Min Episode Reward）: 256.43
🎉 Walker2d 训练效果良好！Walker 能较稳定地行走。


In [7]:
import gymnasium as gym
import numpy as np
import cv2
from stable_baselines3 import PPO

# 加载训练好的 PPO 模型
model = PPO.load("walker2d_ppo")

# 创建环境（启用录像）
env = gym.make("Walker2d-v5", render_mode="rgb_array")

# 录制视频
video_frames = []
n_episodes = 5  # 录制 5 轮

episode_rewards = []
episode_lengths = []

for episode in range(n_episodes):
    obs, _ = env.reset()
    done = False
    total_reward = 0
    steps = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _, _ = env.step(action)
        total_reward += reward
        steps += 1

        # 记录视频帧
        frame = env.render()
        video_frames.append(frame)

    episode_rewards.append(total_reward)
    episode_lengths.append(steps)

# 释放环境
env.close()

# 保存视频
video_path = "walker2d_run.mp4"

# 将帧数据写入 MP4 文件
height, width, _ = video_frames[0].shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_path, fourcc, 30, (width, height))

for frame in video_frames:
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

out.release()

# 计算 Walker2d 运行的统计数据
mean_reward = np.mean(episode_rewards)
std_reward = np.std(episode_rewards)
mean_length = np.mean(episode_lengths)

print(f"🏆 Walker2d 运行视频已录制，视频文件：{video_path}")
print(f"✅ 平均奖励（Mean Reward）: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"✅ 平均步数（Mean Episode Length）: {mean_length:.2f} steps")
print(f"✅ 单轮最大奖励（Max Episode Reward）: {np.max(episode_rewards):.2f}")
print(f"✅ 单轮最小奖励（Min Episode Reward）: {np.min(episode_rewards):.2f}")


🏆 Walker2d 运行视频已录制，视频文件：walker2d_run.mp4
✅ 平均奖励（Mean Reward）: 264.23 ± 3.37
✅ 平均步数（Mean Episode Length）: 146.40 steps
✅ 单轮最大奖励（Max Episode Reward）: 268.63
✅ 单轮最小奖励（Min Episode Reward）: 259.69
